In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 数据准备

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
train_data = pd.read_csv(r'../input/fortrainvalidation/Train_Merge_Data.csv')
validation_data = pd.read_csv(r'../input/fortrainvalidation/Validation_Merge_Data.csv')

In [ ]:
# train_data = shuffle(train_data)
# validation_data = shuffle(validation_data)

In [ ]:
train_label_Sbp = train_data.iloc[:,635]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,636]
train_label_Dbp = train_label_Dbp.values

train_data = train_data.iloc[:,:625]
train_data = train_data.values

In [ ]:
validation_label_Sbp = validation_data.iloc[:,635]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,636]
validation_label_Dbp = validation_label_Dbp.values

validation_data = validation_data.iloc[:,:625]
validation_data = validation_data.values

In [ ]:
print("train_data_information:")
print(train_data.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)
print("validation_data_information:")
print(validation_data.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

# 模型

In [ ]:
inputs = keras.Input(shape=(625,1))

x=32

In [ ]:
conv1 = layers.Conv1D(x,3,padding='same')(inputs)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation(tf.nn.relu)(conv1)

conv1 = layers.Conv1D(x,3,padding='same')(conv1)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation(tf.nn.relu)(conv1)

pool1 = layers.MaxPooling1D(pool_size=2)(conv1)

In [ ]:
conv2 = layers.Conv1D(x*2,3,padding='same')(pool1)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation(tf.nn.relu)(conv2)

conv2 = layers.Conv1D(x*2,3,padding='same')(conv2)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation(tf.nn.relu)(conv2)

pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

In [ ]:
conv3 = layers.Conv1D(x*4,3,padding='same')(pool2)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation(tf.nn.relu)(conv3)

conv3 = layers.Conv1D(x*4,3,padding='same')(conv3)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation(tf.nn.relu)(conv3)

pool3 = layers.MaxPooling1D(pool_size=2)(conv3)

In [ ]:
conv4 = layers.Conv1D(x*8,3,padding='same')(pool3)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation(tf.nn.relu)(conv4)

conv4 = layers.Conv1D(x*8,3,padding='same')(conv4)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation(tf.nn.relu)(conv4)

pool4 = layers.MaxPooling1D(pool_size=2)(conv4)

In [ ]:
conv5 = layers.Conv1D(x*16,3,padding='same')(pool4)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation(tf.nn.relu)(conv5)

conv5 = layers.Conv1D(x*16,3,padding='same')(conv5)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation(tf.nn.relu)(conv5)

In [ ]:
up6 = layers.Concatenate(axis=2)([layers.UpSampling1D(size=2)(conv5),conv4])

conv6 = layers.Conv1D(x*8,3,padding='same')(up6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation(tf.nn.relu)(conv6)

conv6 = layers.Conv1D(x*8,3,padding='same')(conv6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation(tf.nn.relu)(conv6)

In [ ]:
up7 = layers.Concatenate(axis=2)([layers.UpSampling1D(size=2)(conv6),conv3])

conv7 = layers.Conv1D(x*4,3,padding='same')(up7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation(tf.nn.relu)(conv7)

conv7 = layers.Conv1D(x*4,3,padding='same')(conv7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation(tf.nn.relu)(conv7)

In [ ]:
up8 = layers.Concatenate(axis=2)([layers.UpSampling1D(size=2)(conv7),conv2])

conv8 = layers.Conv1D(x*2,3,padding='same')(up8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation(tf.nn.relu)(conv8)

conv8 = layers.Conv1D(x*2,3,padding='same')(conv8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation(tf.nn.relu)(conv8)
conv8 = layers.UpSampling1D(size=2)(conv8)

conv8 = layers.ZeroPadding1D((0,1))(conv8)

In [ ]:
up9 = layers.Concatenate(axis=2)([conv8,conv1])

conv9 = layers.Conv1D(x,3,padding='same')(up9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation(tf.nn.relu)(conv9)

conv9 = layers.Conv1D(x,3,padding='same')(conv9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation(tf.nn.relu)(conv9)


conv10 = layers.Conv1D(1,1)(conv9)

In [ ]:
conv10 =layers.Flatten()(conv10)

In [ ]:
com_layer = layers.Dense(32,activation='relu')(conv10)

com_layer = layers.Dense(32,activation='relu')(com_layer)


In [ ]:
outputs_sbp = layers.Dense(1,name='Sbp')(com_layer)
outputs_dbp = layers.Dense(1,name='Dbp')(com_layer)

model = keras.Model(inputs=inputs,outputs=[outputs_sbp,outputs_dbp])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='./model_show.png', show_shapes=True, show_layer_names=True)

# 基础参数设置

In [ ]:
BATCH_SIZE = 128

# 训练自定义模块

In [ ]:
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

In [ ]:
checkpoint_filepath = r'./Model_UN_01.h5'
Save_epochs = 5 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_loss',
    mode='min',
    save_best_only=True)

In [ ]:
# the_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001,decay=0.0001)
the_optimizer = keras.optimizers.Adam(lr=0.0001)

# 编译模型

In [ ]:
# model.compile(loss=keras.losses.mse, optimizer=the_optimizer,metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])
model.compile(loss={'Sbp':"mae",'Dbp':"mae"}, optimizer=the_optimizer,metrics=[standard_deviation])

# 训练模型

In [ ]:
history = model.fit(train_data,{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=BATCH_SIZE,
                    epochs=5,
                    callbacks=model_checkpoint_callback,
                    validation_data=(validation_data,{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
model.save(r'./Last_Model_01.h5')

# 加载模型

In [ ]:
def Conv1d_layer(inputs,Filters,Kernel_size,Padding='same',Strides=1):
    com_layer = layers.Conv1D(filters=Filters,kernel_size=Kernel_size,padding=Padding,strides=Strides)(inputs)
    com_layer = layers.BatchNormalization()(com_layer)
    com_layer = layers.Activation(tf.nn.relu)(com_layer)
    return com_layer

In [ ]:
#短序列-目前初始输入维度为（-1,125，1）
def Resnet_Model_S(inputs):
    #卷积块1
    com_layer = Conv1d_layer(inputs,Filters=64,Kernel_size=8,Padding='same',Strides=1)
    #卷积块2
    com_layer = Conv1d_layer(com_layer,Filters=128,Kernel_size=5,Padding='same',Strides=1)
    #卷积块3
    com_layer = layers.Conv1D(filters=128,kernel_size=3,padding='same',strides=1)(com_layer)
    #批标准化
    com_layer = layers.BatchNormalization()(com_layer)
    add_layer = layers.BatchNormalization()(inputs)
    #信息叠加
    com_layer = layers.add([add_layer,com_layer])
    
    return com_layer

In [ ]:
#长序列-目前初始输入维度为（625，1）
def Resnet_Model_L(inputs):
    #卷积块1
    com_layer = Conv1d_layer(inputs,Filters=64,Kernel_size=8,Padding='same',Strides=1)
    #卷积块2
    com_layer = Conv1d_layer(com_layer,Filters=128,Kernel_size=5,Padding='same',Strides=1)
    #卷积块3
    com_layer = layers.Conv1D(filters=128,kernel_size=3,padding='same',strides=1)(com_layer)
    #批标准化
    com_layer = layers.BatchNormalization()(com_layer)
    add_layer = layers.BatchNormalization()(inputs)
    #信息叠加
    com_layer = layers.add([add_layer,com_layer])
    
    return com_layer

In [ ]:
def Separate_Data(inputs,num_of_cycle):
    per_input = inputs[:,125*(num_of_cycle-1):125*num_of_cycle]
    return per_input

In [ ]:
def Separate_Data_For_Conv1d(per_input):
    per_input = tf.reshape(per_input,(-1,125,1))
    return per_input

In [ ]:
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

In [ ]:
model = keras.models.load_model(r'../input/model-res-gru-650/Model_ls_650_03.h5',           
                                    custom_objects={
                                    'Conv1d_layer':Conv1d_layer,
                                    'Separate_Data':Separate_Data,
                                    'Separate_Data_For_Conv1d':Separate_Data_For_Conv1d,
                                    'Resnet_Model_L':Resnet_Model_L,
                                    'Resnet_Model_S':Resnet_Model_S,
                                    
                                    
                                    'standard_deviation':standard_deviation
                                    
                                    }
                               )

In [ ]:
model = keras.models.load_model(r'./Last_Model_01.h5',           
                                    custom_objects={
    
                                    
                                    'standard_deviation':standard_deviation,
                                    }
                               )

In [ ]:
test_data = pd.read_csv(r'../input/fortrainvalidation/Validation_Merge_Data.csv')

In [ ]:
test_label_Sbp = test_data.iloc[:,635]
test_label_Sbp = test_label_Sbp.values

test_label_Dbp = test_data.iloc[:,636]
test_label_Dbp = test_label_Dbp.values

test_data = test_data.iloc[:,:625]
test_data = test_data.values

In [ ]:
sbp_pre_0,dbp_pre_0 = model.predict(test_data)

In [ ]:
np.mean(abs(dbp_pre_0-test_label_Dbp))

In [ ]:
np.mean(abs(sbp_pre_0-test_label_Sbp))

In [ ]:
sbp_true = test_label_Sbp
dbp_true = test_label_Dbp

In [ ]:
sbp_pre= sbp_pre_0
dbp_pre= dbp_pre_0

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(sbp_pre,'r.-',label='SBP_pred')

plt.plot(sbp_true,'b.-',label='SBP_true')
plt.plot(dbp_pre,'g.-',label='DBP_pred')

plt.plot(dbp_true,'y.-',label='DBP_true')
plt.title('Predict Display',fontsize=16)
plt.xlabel('Numbers',fontsize=14)
x_major_locator=MultipleLocator(10)
y_major_locator=MultipleLocator(5)
ax=plt.gca()
ax.xaxis.set_major_locator(x_major_locator)
ax.yaxis.set_major_locator(y_major_locator)

plt.tick_params(labelsize=10)

plt.ylim(25,220)
plt.ylabel('mmHg',fontsize=14)
plt.legend(fontsize=8)
plt.show()